In [1]:
from quantrocket.history import create_usstock_db
create_usstock_db("usstock-free-1d", bar_size="1 day", free=True)

{'status': 'successfully created quantrocket.v2.history.usstock-free-1d.sqlite'}

In [14]:
from quantrocket.history import collect_history
collect_history("usstock-free-1d")

{'status': 'the historical data will be collected asynchronously'}

In [1]:
from quantrocket.master import get_securities
# by specifying sec_types='STK', we exclude ETFs, which are present in the sample data 
securities = get_securities(vendors="usstock", sec_types="STK")
securities.head()

,Symbol,Exchange,Country,Currency,SecType,Etf,Timezone,Name,PriceMagnifier,Multiplier,Delisted,DateDelisted,LastTradeDate,RolloverDate
Sid,,,,,,,,,,,,,,
FIBBG000B9XRY4,AAPL,XNAS,US,USD,STK,False,America/New_York,APPLE INC,1,1,False,NaT,NaT,NaT
FIBBG000BFWKC0,MON,XNYS,US,USD,STK,False,America/New_York,MONSANTO CO,1,1,True,2018-06-06,NaT,NaT
FIBBG000BKZB36,HD,XNYS,US,USD,STK,False,America/New_York,HOME DEPOT INC,1,1,False,NaT,NaT,NaT
FIBBG000BMHYD1,JNJ,XNYS,US,USD,STK,False,America/New_York,JOHNSON & JOHNSON,1,1,False,NaT,NaT,NaT
FIBBG000BPH459,MSFT,XNAS,US,USD,STK,False,America/New_York,MICROSOFT CORP,1,1,False,NaT,NaT,NaT


In [17]:
from quantrocket.master import create_universe
create_universe("usstock-free", sids=securities.index.tolist(),replace=True)

{'code': 'usstock-free', 'provided': 8, 'inserted': 8, 'total_after_insert': 8}

In [18]:
filtered_securities = securities[securities.Delisted==False]
# also limit the number of columns for readability
filtered_securities = filtered_securities[["Symbol", "Exchange", "Name", "Delisted"]]
filtered_securities.head()

,Symbol,Exchange,Name,Delisted
Sid,,,,
FIBBG000B9XRY4,AAPL,XNAS,APPLE INC,False
FIBBG000BKZB36,HD,XNYS,HOME DEPOT INC,False
FIBBG000BMHYD1,JNJ,XNYS,JOHNSON & JOHNSON,False
FIBBG000BPH459,MSFT,XNAS,MICROSOFT CORP,False
FIBBG000GZQ728,XOM,XNYS,EXXON MOBIL CORP,False


In [19]:
create_universe("usstock-free-active", sids=filtered_securities.index.tolist())

{'code': 'usstock-free-active',
 'provided': 6,
 'inserted': 6,
 'total_after_insert': 6}

In [2]:
import pandas as pd
from quantrocket import get_prices

# Get closing prices
prices = get_prices("usstock-free-1d", universes="usstock-free", start_date="2023-01-01", end_date="2023-12-31", exclude_sids=['FIBBG000BKZB36', 'FIBBG000BMHYD1', 'FIBBG000BPH459', 'FIBBG000BPH459', 'FIBBG000GZQ728', 'FIBBG00B3T3HD3'], fields=["Close"])
cl = [0] #Since we don't consider the past data for the first day we consider its returns as 0
s = [0] #Since we don't consider the past data for the first day we consider its state as 0
print(len(prices))
for i in range(1, len(prices)):
    p_prev = prices.iloc[i - 1]
    p_curr = prices.iloc[i]
    r = (p_curr - p_prev) / p_prev
    if r.values[0] >= 0.01:
        s.append(1)  # Bull
    elif r.values[0] > -0.01:
        s.append(0)  # Flat
    else:
        s.append(-1)  # Bear
    cl.append(r.values[0])

# Calculate returns
prices['Returns'] = cl
prices['State'] = s
# State the dataframe
prices



250


Sid               FIBBG000B9XRY4   Returns  State
Field Date                                       
Close 2023-01-03        124.2163  0.000000      0
      2023-01-04        125.4975  0.010314      1
      2023-01-05        124.1666 -0.010605     -1
      2023-01-06        128.7352  0.036794      1
      2023-01-09        129.2616  0.004089      0
...                          ...       ...    ...
      2023-12-22        193.3533 -0.005547      0
      2023-12-26        192.8040 -0.002841      0
      2023-12-27        192.9038  0.000518      0
      2023-12-28        193.3333  0.002226      0
      2023-12-29        192.2846 -0.005424      0

[250 rows x 3 columns]

In [3]:




def transition(price):
    transition1 = [[-1,-1,0],[0,-1,0],[1,-1,0],[-1,0,0],[0,0,0],[1,0,0],[-1,1,0],[0,1,0],[1,1,0]]
    for i in range(1,len(price)):
        if price[i-1] == -1 and price[i]==-1:
            transition1[0][2] += 1
        elif price[i-1] == 0 and price[i]==-1:
            transition1[1][2] += 1
        elif price[i-1] == 1 and price[i]==-1:
            transition1[2][2] += 1
        elif price[i-1] == -1 and price[i]==0:
            transition1[3][2] += 1
        elif price[i-1] == 0 and price[i]==0:
            transition1[4][2] += 1
        elif price[i-1] == 1 and price[i]==0:
            transition1[5][2] += 1
        elif price[i-1] == -1 and price[i]==1:
            transition1[6][2] += 1
        elif price[i-1] == 0 and price[i]==1:
            transition1[7][2] += 1
        else:
            transition1[8][2] += 1
    return transition1
transition1 = transition(prices['State'])    
# Print the transition matrix in the desired format
print("| FROM | TO   | COUNT|")
print("|------|------|------|")
for row in transition1:
  from_state, to_state, count = row
  print(f"| {from_state:<3}  | {to_state:<3}  | {count:<3}  |")
'''
              FROM
             -1  0  1  
            ___________
        -1 |
 TO      0 |
         1 |


'''
def matrix(transition1):
    matrix = [[0,0,0],[0,0,0],[0,0,0]]
    matrix[0][0] = transition1[0][2]
    matrix[0][1] = transition1[1][2]
    matrix[0][2] = transition1[2][2]
    matrix[1][0] = transition1[3][2]
    matrix[1][1] = transition1[4][2]
    matrix[1][2] = transition1[5][2]
    matrix[2][0] = transition1[6][2]
    matrix[2][1] = transition1[7][2]
    matrix[2][2] = transition1[8][2]
    return matrix

matrixs = matrix(transition1)
print('Matrix')
for i in matrixs:
    print(i)



| FROM | TO   | COUNT|
|------|------|------|
| -1   | -1   | 5    |
| 0    | -1   | 23   |
| 1    | -1   | 7    |
| -1   | 0    | 26   |
| 0    | 0    | 94   |
| 1    | 0    | 38   |
| -1   | 1    | 4    |
| 0    | 1    | 41   |
| 1    | 1    | 11   |
Matrix
[5, 23, 7]
[26, 94, 38]
[4, 41, 11]


In [11]:
def prob_matrix(matrix):
    matrix1 = [[0,0,0],[0,0,0],[0,0,0]]
    for i in range(len(matrix)):
        for j in range(len(matrix)):
            s = sum(matrix[i])
            p = matrix[i][j]/s
            matrix1[i][j] = p
    return matrix1
            
prob = prob_matrix(matrixs)
for i in prob:
    print(i)

[0.14285714285714285, 0.6571428571428571, 0.2]
[0.16455696202531644, 0.5949367088607594, 0.24050632911392406]
[0.07142857142857142, 0.7321428571428571, 0.19642857142857142]


In [8]:
def portfolio_value(price):
  portfolio_val = 0  # Initialize portfolio_val to 0 within the function
  max_portfolio_val = 0
  pf = [0] # Initialize portfolio_val to 0 for the first state
  mpf = [0] # Initialize max_portfolio_val to 0 for the first state
  buy = []
  for i in range(1, len(price)):
    if price[i-1] == 0 and price[i] == 1:
        portfolio_val += 1
        max_portfolio_val += 1
        buy.append(i)
        pf.append(portfolio_val)
        mpf.append(max_portfolio_val)
    elif price[i-1] == 0 and price[i] == -1:
        portfolio_val -= 1
        pf.append(portfolio_val)
        mpf.append(max_portfolio_val)
    else:
        pf.append(portfolio_val)
        mpf.append(max_portfolio_val)
  count = 0
  for i in range(1, len(price)):
    if price[i-1] == 0:
        count+=1
  print(count)
  return portfolio_val,pf,buy,mpf,max_portfolio_val
pval,p,b,mp,mpval = portfolio_value(prices['State'])
prices['V(N)'] = p
prices['max_V(N)'] = mp 
prices

158


Sid               FIBBG000B9XRY4   Returns  State  max_V(N)  V(N)
Field Date                                                       
Close 2023-01-03        124.2163  0.000000      0         0     0
      2023-01-04        125.4975  0.010314      1         1     1
      2023-01-05        124.1666 -0.010605     -1         1     1
      2023-01-06        128.7352  0.036794      1         1     1
      2023-01-09        129.2616  0.004089      0         1     1
...                          ...       ...    ...       ...   ...
      2023-12-22        193.3533 -0.005547      0        41    18
      2023-12-26        192.8040 -0.002841      0        41    18
      2023-12-27        192.9038  0.000518      0        41    18
      2023-12-28        193.3333  0.002226      0        41    18
      2023-12-29        192.2846 -0.005424      0        41    18

[250 rows x 5 columns]

In [14]:
print("The state of First day is considered as Flat(0) as we don't consider the previous data")
print()
print("Transition Table:")
# Print the transition matrix in the desired format
print("| FROM | TO   | COUNT|")
print("|------|------|------|")
for row in transition1:
  from_state, to_state, count = row
  print(f"| {from_state:<3}  | {to_state:<3}  | {count:<3}  |")
print()
print('Transition Matrix')
for i in matrixs:
    print(i)
print()
print("Probability Distribution Matrix:")
for i in prob:
    print(i)
print()

dates = prices.index.levels[1]
date_list = []
for i in range(0,len(dates)):
    date_string = dates[i].strftime('%Y-%m-%d')  # Format as a string (YYYY-MM-DD)
    date_list.append(date_string)
#print(date_list)
print("Normal Portfolio Value") # Considering both ups and downs in a market
print("Portfolio Value:",pval)
print()

print("To maximize Portfolio value by only buying in bull market")#Under the assumption to only increase the portfolio value
print("Portfolio Value:",len(b))#Since b list has the indeces of the days we buy only in bull market
print()
print("Buy Dates or Indices:")
for i in b:
    print(date_list[i],' -->',i)

The state of First day is considered as Flat(0) as we don't consider the previous data

Transition Table:
| FROM | TO   | COUNT|
|------|------|------|
| -1   | -1   | 5    |
| 0    | -1   | 23   |
| 1    | -1   | 7    |
| -1   | 0    | 26   |
| 0    | 0    | 94   |
| 1    | 0    | 38   |
| -1   | 1    | 4    |
| 0    | 1    | 41   |
| 1    | 1    | 11   |

Transition Matrix
[5, 23, 7]
[26, 94, 38]
[4, 41, 11]

Probability Distribution Matrix:
[0.14285714285714285, 0.6571428571428571, 0.2]
[0.16455696202531644, 0.5949367088607594, 0.24050632911392406]
[0.07142857142857142, 0.7321428571428571, 0.19642857142857142]

Normal Portfolio Value
Portfolio Value: 18

To maximize Portfolio value by only buying in bull market
Portfolio Value: 41

Buy Dates or Indices:
2023-01-04  --> 1
2023-01-11  --> 6
2023-01-13  --> 8
2023-01-20  --> 12
2023-01-26  --> 16
2023-02-02  --> 21
2023-02-13  --> 28
2023-02-15  --> 30
2023-03-03  --> 41
2023-03-16  --> 50
2023-03-20  --> 52
2023-03-29  --> 59
2023-03-